# Parse tables to database

https://nbviewer.jupyter.org/github/FinanceData/OpenDartReader/blob/master/docs/OpenDartReader_reference_manual.ipynb


```
KOSPI
├── Financial statement(Balance Sheet)
├── Comprehensive Income Statement
├── Cash flow statement
└── Statement of Changes in Equity
```

In [ ]:
from collections import defaultdict
from tqdm import tqdm
import sys
import sqlite3
from pathlib import Path

class DatabaseProcessor(object):
    def __init__(self, base_path, **kwargs):
        """
        params
         - base_path
         - info_columns
         - data_columns
        """
        self.info_columns = kwargs["info_columns"]
        self.data_columns = kwargs["data_columns"]
        self.update_person_columns = kwargs["update_person_columns"]
        db_path = Path(base_path) / "database"
        if not db_path.exists():
            db_path.mkdir()
        self.db_path = db_path / "lecture.db"
        self.check_is_table = lambda d: len(set(list(map(lambda x: len(x), d.values())))) <= 1
        self.update_src_dict_cols = ('lasttable_id', 'lec_ids', 'pages', 'lastupdate', 'ids')
        
    def create_connection(self):
        try:
            conn = sqlite3.connect(str(self.lec_db_path))
            return conn
        except:
            print("Unexpected error:", sys.exc_info()[0])
            raise
    
    def create_table(self):
        """
        table columns:
         - lecinfo: ('id', 'page', 'name', 'date', 'prof', 'partner', 'imglink', 'access')
         - lecdata: ('id', 'lec_id', 'person', 'daynum', 'daylast', 'dayenroll')
        """
        lec_table = """CREATE TABLE lecinfo (
            {} integer PRIMARY KEY,
            {} text,
            {} text,
            {} text,
            {} text,
            {} text,
            {} text,
            {} integer
            ); """.format(*self.info_columns)

        data_table = """CREATE TABLE lecdata (
            {} integer PRIMARY KEY,
            {} integer,
            {} text,
            {} integer,
            {} text,
            {} text,
            FOREIGN KEY(lec_id) REFERENCES lecinfo(id)
            ); """.format(*self.data_columns)
        conn = self.create_connection()
        with conn:
            self._ct(conn, lec_table)
            self._ct(conn, data_table)
        print("created tables")
    
    def delete_table(self):
        conn = self.create_connection()
        with conn:
            self._dt(conn)
        print("deleted tables")
    
    def check_table_info(self, table_name):
        conn = self.create_connection()
        with conn:
            c = conn.cursor()
            res = c.execute(f"PRAGMA table_info({table_name});")
            desc = list(zip(*res.description))[0]
            print("\t".join(desc))
            for r in res.fetchall():
                print("\t".join(list(map(lambda x: str(x), r))))

In [26]:
from collections import defaultdict
from tqdm import tqdm
import sys
import sqlite3
from pathlib import Path

In [ ]:
base_path = "."
db_path = Path(base_path) / "database"

# Finance DataReader

In [2]:
from private.apikey import APIKEY

import pandas as pd
import OpenDartReader
import FinanceDataReader as fdr

dart = OpenDartReader(APIKEY) 

In [12]:
stocks = fdr.StockListing("KOSPI")
stocks = stocks.loc[~stocks["Sector"].isnull(), :]
stocks_syms = stocks["Symbol"].values
stocks = stocks.reset_index(drop=True)

In [13]:
stocks

,Symbol,Market,Name,Sector,Industry,ListingDate,SettleMonth,Representative,HomePage,Region
0,095570,KOSPI,AJ네트웍스,산업용 기계 및 장비 임대업,"렌탈(파렛트, OA장비, 건설장비)",2015-08-21,12월,박대현,http://www.ajnet.co.kr,서울특별시
1,006840,KOSPI,AK홀딩스,기타 금융업,지주사업,1999-08-11,12월,"채형석, 이석주(각자 대표이사)",http://www.aekyunggroup.co.kr,서울특별시
2,027410,KOSPI,BGF,기타 금융업,지주회사,2014-05-19,12월,홍정국,http://www.bgf.co.kr,서울특별시
3,282330,KOSPI,BGF리테일,종합 소매업,체인화 편의점,2017-12-08,12월,이건준,http://www.bgfretail.com,서울특별시
4,138930,KOSPI,BNK금융지주,기타 금융업,금융지주회사,2011-03-30,12월,김지완,http://www.bnkfg.com,부산광역시
...,...,...,...,...,...,...,...,...,...,...
800,079980,KOSPI,휴비스,화학섬유 제조업,"합성섬유(폴리에스테르원사,원면),재생섬유,폴리에스텔 원사,원면,고상칩 제조,도소매",2012-02-23,12월,신유동,http://www.huvis.com,서울특별시
801,005010,KOSPI,휴스틸,1차 철강 제조업,"강관(배관용,구조용,유정용) 제조,도매",1973-06-29,12월,박훈,http://www.husteel.com,서울특별시
802,069260,KOSPI,휴켐스,기타 화학제품 제조업,"화합물,화학제품 제조",2002-10-07,12월,신진용,http://www.huchems.com,서울특별시
803,000540,KOSPI,흥국화재,보험업,손해보험,1974-12-05,12월,권중원,http://www.insurance.co.kr,서울특별시


In [15]:
import sqlite3
from pathlib import Path

db_path = Path("./private/")
con = sqlite3.connect(db_path / "kospi.db")

In [16]:
stocks.to_sql("kospi", con)

## Company Basic

- corp_name: 정식명칭
- corp_name_eng: 영문명칭
- stock_name: 종목명 또는 약식명칭 
- stock_code: 상장회사인 경우 주식의 종목코드
- ceo_nm: 대표자명
- crop_cls: 법인구분
- jurir_no: 법인등록번호
- bizr_no: 사업자등록번호
- adres: 주소
- hm_url: 홈페이지
- ir_url: IR홈페이지
- phn_no: 전화번호
- fax_no: 팩스번호
- induty_code: 업종코드
- estdt: 설립일
- acc_mt: 결산월

In [ ]:
from tqdm.notebook import tqdm

In [19]:
basic_cols = [
    'corp_name', 'corp_name_eng', 'stock_name', 'stock_code', 'ceo_nm', 'corp_cls', 
    'jurir_no', 'bizr_no', 'adres', 'hm_url', 'ir_url', 'phn_no', 'fax_no', 'induty_code', 'est_dt', 'acc_mt'
]
df_basic = pd.DataFrame([dart.company(s) for s in tqdm(stocks_syms)]).loc[:, basic_cols]

In [22]:
df_basic.to_sql("company", con)

## Report

- corp (문자열): 검색대상 회사의 종목코드를 지정합니다. 고유번호, 회사이름도 가능합니다.
- key_word (문자열): 조회 내용 지정, 아래 "key_word 항목"을 참고하십시오 ('증자','배당','자기주식','최대주주','최대주주변동','소액주주','임원','직원','임원개인보수','임원전체보수','개인별보수','타법인출자')
- bsns_year (문자열 혹은 정수값): 사업연도
- reprt_code (문자열): 보고서 코드 ('11013'=1분기보고서, '11012'=반기보고서, '11014'=3분기보고서, '11011'=사업보고서)

반환값 (DataFrame): 조회 결과를 데이터프레임(DataFrame)으로 반환합니다. 데이터프레임의 각 컬럼은 다음과 같습니다.

- rcept_no: 접수번호
- corp_cls: 법인구분 Y(유가), K(코스닥), N(코넥스), E(기타)
- corp_code: 고유번호
- corp_name: 법인명

key_word 항목 지정에 따라 결과 데이터의 컬럼이 달라집니다. '배당' - 배당에 관한 사항

- se: 구분. 유상증자(주주배정), 전환권행사 등
- stock_knd: 주식 종류
- thstrm: 당기
- frmtrm: 전기
- lwfr: 전전기

In [48]:
dart.company("삼성전자")

{'status': '000',
 'message': '정상',
 'corp_code': '00126380',
 'corp_name': '삼성전자(주)',
 'corp_name_eng': 'SAMSUNG ELECTRONICS CO,.LTD',
 'stock_name': '삼성전자',
 'stock_code': '005930',
 'ceo_nm': '김기남, 김현석, 고동진',
 'corp_cls': 'Y',
 'jurir_no': '1301110006246',
 'bizr_no': '1248100998',
 'adres': '경기도 수원시 영통구  삼성로 129 (매탄동)',
 'hm_url': 'www.sec.co.kr',
 'ir_url': '',
 'phn_no': '031-200-1114',
 'fax_no': '031-200-7538',
 'induty_code': '264',
 'est_dt': '19690113',
 'acc_mt': '12'}

In [52]:
corp = '005930'
bsns_year = '2020'

In [59]:
stocks["SettleMonth"].unique()  # 우선은 사업보고서 기준으로 해야할듯

array(['12월', '05월', '11월', '09월', '03월', '06월', '02월', '08월'],
      dtype=object)

In [58]:
# ['증자', '배당', '자기주식', '최대주주', '최대주주변동', '소액주주', '임원', '직원', '임원개인보수', '임원전체보수', '개인별보수', '타법인출자']
dart.report(corp, '임원개인보수', bsns_year, report_code='11011')

,rcept_no,corp_cls,corp_code,corp_name,nm,ofcps,mendng_totamt,mendng_totamt_ct_incls_mendng
0,20210309000744,Y,00126380,삼성전자,최윤호,이사,"3,028,000,000",-
1,20210309000744,Y,00126380,삼성전자,한종희,이사,"4,183,000,000",-
2,20210309000744,Y,00126380,삼성전자,이상훈,사장,"4,594,000,000",-
3,20210309000744,Y,00126380,삼성전자,김현석,대표이사,"5,457,000,000",-
4,20210309000744,Y,00126380,삼성전자,고동진,대표이사,"6,712,000,000",-
5,20210309000744,Y,00126380,삼성전자,김기남,대표이사,"8,274,000,000",-


## Finstate
- corp (문자열): 검색대상 회사의 종목코드를 지정합니다. 고유번호, 회사이름도 가능합니다.
- bsns_year (문자열 혹은 정수값): 사업연도
- reprt_code (문자열): 보고서 코드 ('11013'=1분기보고서, '11012'=반기보고서, '11014'=3분기보고서, '11011'=사업보고서)

반환값 (DataFrame): 조회 결과를 데이터프레임(DataFrame)으로 반환합니다. 데이터프레임의 각 컬럼은 다음과 같습니다.

- rcept_no: 접수번호
- corp_code: 사업 연도
- stock_code: 종목 코드
- reprt_code: 보고서 코드
- account_nm: 계정명 (예: 자본총계)
- fs_div: 개별/연결구분 ('CFS'=연결재무제표, 'OFS'=재무제표)
- fs_nm: 개별/연결명 ('연결재무제표' 또는 '재무제표')
- sj_div: 재무제표구분 ('BS'=재무상태표, 'IS'=손익계산서)
- sj_nm: 재무제표명 ( '재무상태표' 또는 '손익계산서')
- thstrm_nm: 당기명
- thstrm_dt: 당기일자
- thstrm_amount: 당기금액
- thstrm_add_amount: 당기누적금액
- frmtrm_nm: 전기명
- frmtrm_dt: 전기일자
- frmtrm_amount: 전기금액
- frmtrm_add_amount: 전기누적금액
- bfefrmtrm_nm: 전전기명
- bfefrmtrm_dt: 전전일자
- bfefrmtrm_amount: 전전기금액
- ord: 계정과목 정렬순서

In [63]:
dart.finstate(corp, bsns_year, reprt_code='11011')

,rcept_no,reprt_code,bsns_year,corp_code,stock_code,fs_div,fs_nm,sj_div,sj_nm,account_nm,thstrm_nm,thstrm_dt,thstrm_amount,frmtrm_nm,frmtrm_dt,frmtrm_amount,bfefrmtrm_nm,bfefrmtrm_dt,bfefrmtrm_amount,ord
0,20210309000744,11011,2020,00126380,005930,CFS,연결재무제표,BS,재무상태표,유동자산,제 52 기,2020.12.31 현재,"198,215,579,000,000",제 51 기,2019.12.31 현재,"181,385,260,000,000",제 50 기,2018.12.31 현재,"174,697,424,000,000",1
1,20210309000744,11011,2020,00126380,005930,CFS,연결재무제표,BS,재무상태표,비유동자산,제 52 기,2020.12.31 현재,"180,020,139,000,000",제 51 기,2019.12.31 현재,"171,179,237,000,000",제 50 기,2018.12.31 현재,"164,659,820,000,000",3
2,20210309000744,11011,2020,00126380,005930,CFS,연결재무제표,BS,재무상태표,자산총계,제 52 기,2020.12.31 현재,"378,235,718,000,000",제 51 기,2019.12.31 현재,"352,564,497,000,000",제 50 기,2018.12.31 현재,"339,357,244,000,000",5
3,20210309000744,11011,2020,00126380,005930,CFS,연결재무제표,BS,재무상태표,유동부채,제 52 기,2020.12.31 현재,"75,604,351,000,000",제 51 기,2019.12.31 현재,"63,782,764,000,000",제 50 기,2018.12.31 현재,"69,081,510,000,000",7
4,20210309000744,11011,2020,00126380,005930,CFS,연결재무제표,BS,재무상태표,비유동부채,제 52 기,2020.12.31 현재,"26,683,351,000,000",제 51 기,2019.12.31 현재,"25,901,312,000,000",제 50 기,2018.12.31 현재,"22,522,557,000,000",9
5,20210309000744,11011,2020,00126380,005930,CFS,연결재무제표,BS,재무상태표,부채총계,제 52 기,2020.12.31 현재,"102,287,702,000,000",제 51 기,2019.12.31 현재,"89,684,076,000,000",제 50 기,2018.12.31 현재,"91,604,067,000,000",11
6,20210309000744,11011,2020,00126380,005930,CFS,연결재무제표,BS,재무상태표,자본금,제 52 기,2020.12.31 현재,"897,514,000,000",제 51 기,2019.12.31 현재,"897,514,000,000",제 50 기,2018.12.31 현재,"897,514,000,000",13
7,20210309000744,11011,2020,00126380,005930,CFS,연결재무제표,BS,재무상태표,이익잉여금,제 52 기,2020.12.31 현재,"271,068,211,000,000",제 51 기,2019.12.31 현재,"254,582,894,000,000",제 50 기,2018.12.31 현재,"242,698,956,000,000",17
8,20210309000744,11011,2020,00126380,005930,CFS,연결재무제표,BS,재무상태표,자본총계,제 52 기,2020.12.31 현재,"275,948,016,000,000",제 51 기,2019.12.31 현재,"262,880,421,000,000",제 50 기,2018.12.31 현재,"247,753,177,000,000",21
9,20210309000744,11011,2020,00126380,005930,CFS,연결재무제표,IS,손익계산서,매출액,제 52 기,2020.01.01 ~ 2020.12.31,"236,806,988,000,000",제 51 기,2019.01.01 ~ 2019.12.31,"230,400,881,000,000",제 50 기,2018.01.01 ~ 2018.12.31,"243,771,415,000,000",23


In [60]:
dart.xbrl_taxonomy('BS1')

,sj_div,bsns_de,account_id,account_nm,label_kor,label_eng,ifrs_ref,data_tp
0,BS1,20180701,ifrs_StatementOfFinancialPositionAbstract,StatementOfFinancialPositionAbstract,재무상태표 [abstract],Statement of financial position [abstract],,NaN
1,BS1,20180701,ifrs_AssetsAbstract,AssetsAbstract,자산 [abstract],Assets [abstract],,NaN
2,BS1,20180701,ifrs_CurrentAssets,CurrentAssets,유동자산,Current assets,K-IFRS 1001 문단 60,X
3,BS1,20180701,ifrs_CashAndCashEquivalents,CashAndCashEquivalents,현금및현금성자산,Cash and cash equivalents,"K-IFRS 1001 문단 54 (9),K-IFRS 1007 문단 45",X
4,BS1,20180701,ifrs_Cash,Cash,현금,Cash,"K-IFRS 1001 문단 54 (9),K-IFRS 1007 문단 45",X
...,...,...,...,...,...,...,...,...
514,BS1,20180701,dart_VoluntaryReserves,VoluntaryReserves,임의적립금,Voluntary Reserves,"K-IFRS 1001 문단 54 (18),K-IFRS 1001 문단 78 (5),K...",X
515,BS1,20180701,dart_RetainedEarningsBeforeAppropriationsAccum...,RetainedEarningsBeforeAppropriationsAccumulate...,미처분이익잉여금(미처리결손금),Retained earnings before appropriations(accumu...,"K-IFRS 1001 문단 54 (18),K-IFRS 1001 문단 78 (5),K...",X
516,BS1,20180701,ifrs_NoncontrollingInterests,NoncontrollingInterests,비지배지분,Non-controlling interests,K-IFRS 1001 문단 54 (17),X
517,BS1,20180701,ifrs_Equity,Equity,자본총계,Total equity,,NaN



## Finstate_all

sj_nm, sj_div
- 재무상태표(BS): Balance Sheet
- 포괄손익계산서(CIS): Comprehensive Income Statement
- 현금흐름표(CF): Cash Flow statement
- 자본변동표(SCE): Statement of Changes in Equity

In [61]:
df_state = dart.finstate_all(corp, bsns_year, '11011')
df_state["sj_nm"].unique()

array(['재무상태표', '손익계산서', '포괄손익계산서', '현금흐름표', '자본변동표'], dtype=object)

In [41]:
df_state.keys()

Index(['rcept_no', 'reprt_code', 'bsns_year', 'corp_code', 'sj_div', 'sj_nm',
       'account_id', 'account_nm', 'account_detail', 'thstrm_nm',
       'thstrm_amount', 'frmtrm_nm', 'frmtrm_amount', 'bfefrmtrm_nm',
       'bfefrmtrm_amount', 'ord', 'thstrm_add_amount'],
      dtype='object')

In [39]:
df_state['sj_div'].unique()

array(['BS', 'CIS', 'CF', 'SCE'], dtype=object)

In [62]:
df_state.loc[df_state['sj_div'] == 'CIS']

,rcept_no,reprt_code,bsns_year,corp_code,sj_div,sj_nm,account_id,account_nm,account_detail,thstrm_nm,thstrm_amount,frmtrm_nm,frmtrm_amount,bfefrmtrm_nm,bfefrmtrm_amount,ord,thstrm_add_amount
72,20210309000744,11011,2020,00126380,CIS,포괄손익계산서,ifrs-full_ProfitLoss,당기순이익(손실),-,제 52 기,26407832000000,제 51 기,21738865000000,제 50 기,44344857000000,0,
73,20210309000744,11011,2020,00126380,CIS,포괄손익계산서,ifrs-full_OtherComprehensiveIncome,기타포괄손익,-,제 52 기,-3673905000000,제 51 기,3016194000000,제 50 기,-12242000000,1,
74,20210309000744,11011,2020,00126380,CIS,포괄손익계산서,dart_OtherComprehensiveIncomeThatWillNotBeRecl...,후속적으로 당기손익으로 재분류되지 않는 포괄손익,-,제 52 기,1788764000000,제 51 기,-50765000000,제 50 기,-656647000000,2,
75,20210309000744,11011,2020,00126380,CIS,포괄손익계산서,-표준계정코드 미사용-,기타포괄손익-공정가치금융자산평가손익,-,제 52 기,2502733000000,제 51 기,1146599000000,제 50 기,-235865000000,3,
76,20210309000744,11011,2020,00126380,CIS,포괄손익계산서,dart_ShareOfOtherComprehensiveIncomeOfAssociat...,관계기업 및 공동기업의 기타포괄손익에 대한 지분,-,제 52 기,5591000000,제 51 기,-16896000000,제 50 기,-10631000000,4,
77,20210309000744,11011,2020,00126380,CIS,포괄손익계산서,dart_OtherComprehensiveIncomeNetOfTaxGainsLoss...,순확정급여부채(자산) 재측정요소,-,제 52 기,-719560000000,제 51 기,-1180468000000,제 50 기,-410151000000,5,
78,20210309000744,11011,2020,00126380,CIS,포괄손익계산서,dart_OtherComprehensiveIncomeThatWillBeReclass...,후속적으로 당기손익으로 재분류되는 포괄손익,-,제 52 기,-5462669000000,제 51 기,3066959000000,제 50 기,644405000000,6,
79,20210309000744,11011,2020,00126380,CIS,포괄손익계산서,dart_ShareOfOtherComprehensiveIncomeOfAssociat...,관계기업 및 공동기업의 기타포괄손익에 대한 지분,-,제 52 기,-48888000000,제 51 기,48649000000,제 50 기,6688000000,7,
80,20210309000744,11011,2020,00126380,CIS,포괄손익계산서,ifrs-full_GainsLossesOnExchangeDifferencesOnTr...,해외사업장환산외환차이,-,제 52 기,-5380375000000,제 51 기,3016499000000,제 50 기,590638000000,8,
81,20210309000744,11011,2020,00126380,CIS,포괄손익계산서,ifrs-full_GainsLossesOnCashFlowHedgesNetOfTax,현금흐름위험회피파생상품평가손익,-,제 52 기,-33406000000,제 51 기,1811000000,제 50 기,47079000000,9,


In [42]:
df_state['account_nm'].unique()

array(['유동자산', '현금및현금성자산', '단기금융상품', '단기투자자산', '매출채권', '기타수취채권', '재고자산',
       '당기법인세자산', '기타유동자산', '기타금융자산', '비유동자산', '종속기업, 관계기업 및 공동기업투자',
       '장기매출채권', '장기투자자산', '유형자산', '사용권자산', '무형자산', '투자부동산', '이연법인세자산',
       '종업원급여자산', '기타비유동자산', '자산총계', '유동부채', '매입채무', '미지급금', '기타지급채무',
       '차입금', '충당부채', '당기법인세부채', '리스부채', '기타유동부채', '기타금융부채', '비유동부채',
       '장기미지급금', '확정급여부채', '이연법인세부채', '기타비유동부채', '부채총계', '지배기업의 소유지분',
       '자본금', '자본잉여금', '기타자본', '기타포괄손익누계액', '이익잉여금', '비지배지분', '자본총계',
       '부채및자본총계', '매출액', '매출원가', '매출총이익', '판매비와관리비', '영업이익', '금융수익',
       '금융비용', '지분법투자 관련 손익', '기타영업외수익', '기타영업외비용', '법인세비용차감전순이익',
       '법인세비용', '당기순이익', '법인세차감후 기타포괄손익', '확정급여제도의 재측정요소', '해외사업장환산외환차이',
       '파생상품평가손익', '관계기업의 기타포괄손익에 대한 지분', '총포괄손익', '지배기업의 소유주지분',
       '기본주당순이익', '희석주당순이익', '영업활동 현금흐름', '영업으로부터 창출된 현금흐름', '이자의 수취',
       '이자의 지급', '배당금의 수취', '법인세의 납부', '투자활동 현금흐름', '단기금융상품의 감소',
       '단기금융상품의 증가', '단기투자자산의 순증감', '기타금융자산의 감소', '기타금융자산의 증가',
       '기타수취채권의 감소', '기타수취

## Major shareholders

- corp (문자열): 검색대상 회사의 종목코드를 지정합니다. 고유번호, 회사이름도 가능합니다.

반환값 (DataFrame): 조회 결과를 데이터프레임(DataFrame)으로 반환합니다. 데이터프레임의 각 컬럼은 다음과 같습니다.

- rcept_no: 접수번호
- rcept_dt: 접수일자
- corp_code: 종목코드
- corp_name: 회사명
- report_tp: 보고구분
- repror: 대표보고자
- stkqy: 보유주식등의 수
- stkqy_irds: 보유주식등의 증감
- stkrt: 보유비율
- stkrt_irds: 보유비율 증감
- ctr_stkqy: 주요체결 주식등의 수
- ctr_stkrt: 주요체결 보유비율
- report_resn: 보고사유

In [23]:
dart.major_shareholders(corp)

,rcept_no,rcept_dt,corp_code,corp_name,report_tp,repror,stkqy,stkqy_irds,stkrt,stkrt_irds,ctr_stkqy,ctr_stkrt,report_resn
0,20190610000061,2019-06-10,00164779,SK하이닉스,약식,"TheCapitalGroupCompanies,Inc.","52,144,494","-4,996,784",7.16,-0.69,-,-,특별관계자 변동
1,20191011000443,2019-10-11,00164779,SK하이닉스,약식,"TheCapitalGroupCompanies,Inc.","47,385,009","-4,759,485",6.51,-0.65,-,-,특별관계자 변동
2,20191111000092,2019-11-11,00164779,SK하이닉스,약식,"TheCapitalGroupCompanies,Inc.","33,018,535","-14,366,474",4.54,-1.97,-,-,보유비율 변동
3,20191209000274,2019-12-09,00164779,SK하이닉스,약식,BlackRockFundAdvisors,"29,241,942","-7,769,748",4.02,-1.06,-,-,- 투자자금 회수목적으로 장내에서 주식 매도
4,20200106000184,2020-01-06,00164779,SK하이닉스,약식,국민연금공단,"73,745,316","7,510,703",10.13,1.03,-,-,단순추가취득/처분
5,20200207000652,2020-02-07,00164779,SK하이닉스,약식,국민연금공단,"74,062,708","317,392",10.17,0.04,-,-,단순투자목적에서 일반투자목적으로 보유목적 변경
6,20200326001285,2020-03-26,00164779,SK하이닉스,일반,SK텔레콤,"146,299,213","-109,444",20.10,-0.01,-,-,특별관계자 및 보유주식 수 변동
7,20200507000040,2020-05-07,00164779,SK하이닉스,약식,국민연금공단,"81,566,082","7,503,374",11.20,1.03,-,-,단순추가취득/처분


In [24]:
dart.major_shareholders_exec(corp)

,rcept_no,rcept_dt,corp_code,corp_name,repror,isu_exctv_rgist_at,isu_exctv_ofcps,isu_main_shrholdr,sp_stock_lmp_cnt,sp_stock_lmp_irds_cnt,sp_stock_lmp_rate,sp_stock_lmp_irds_rate
0,20190429000447,2019-04-29,00164779,SK하이닉스,최창언,비등기임원,연구위원,-,784,784,0.00,0.00
1,20190625000283,2019-06-25,00164779,SK하이닉스,박근우,비등기임원,상무,-,"1,001","1,000",0.00,0.00
2,20190717000574,2019-07-18,00164779,SK하이닉스,이재형,비등기임원,전무,-,200,200,0.00,0.00
3,20191206000340,2019-12-06,00164779,SK하이닉스,국민연금공단,-,-,10%이상주주,"72,848,416","72,848,416",10.01,10.01
4,20191227000681,2019-12-27,00164779,SK하이닉스,차진석,비등기임원,담당,-,0,-241,0.00,0.00
5,20200102000276,2020-01-02,00164779,SK하이닉스,김정수,비등기임원,담당,-,86,86,0.00,0.00
6,20200102000298,2020-01-02,00164779,SK하이닉스,이일우,비등기임원,담당,-,324,324,0.00,0.00
7,20200102000286,2020-01-02,00164779,SK하이닉스,류성수,비등기임원,담당,-,80,80,0.00,0.00
8,20200103000495,2020-01-03,00164779,SK하이닉스,서강봉,비등기임원,담당,-,"1,080","1,080",0.00,0.00
9,20200106000342,2020-01-06,00164779,SK하이닉스,정우식,비등기임원,담당,-,140,140,0.00,0.00
